In [ ]:
import sys
import os
from pathlib import Path
import tensorflow as tf
import numpy as np
import shutil
import time

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.architecture as ARCH
import SnowDepth.optimal_features as OF
from SnowDepth.config import HOLDOUT_AOI
from SnowDepth.config import FEATURE_NAMES

In [ ]:
# Select holdout AOI
holdout_aoi = HOLDOUT_AOI

# H5 directory and expected output files
h5_dir = ROOT / "data" / "h5_dir"
h5_path_HSIC = h5_dir / "HSIC" / "data_HSIC.h5"
h5_path_PCC  = h5_dir / "PCC"  / "data_PCC.h5"
h5_path_MI   = h5_dir / "MI"   / "data_MI.h5"

# If we have already written H5 files, we can skip a lot of steps here:
if all(p.exists() for p in [h5_path_HSIC, h5_path_PCC, h5_path_MI]):
    print("All H5 files already exist. Skipping feature selection and H5 generation.")
else:
    print("H5 files missing. Running feature selection and writing new H5 files")

    # SET VARIABLES HERE:
    
    # Path to TIFF files
    data_dir = ROOT / "data" / "tif_files"

    # Number of features to select
    top_k = 10

    # Load dataframe
    df = DL.build_df(str(data_dir), drop_invalid=True, upper_threshold=3)
    dev_df = df[df["aoi_name"] != holdout_aoi].copy()

    # Run feature selection
    ff_algos = OF.optimal_feature_sets(dev_df, top_k=top_k, n_per_aoi=10000)

    # Write one H5 file per feature-set
    for name, feats in ff_algos.items():
        out_dir = h5_dir / name
        out_dir.mkdir(parents=True, exist_ok=True)
        DL.build_h5(
            data_dir=str(data_dir),
            out_dir=str(out_dir),
            write_mask=True,
            upper_threshold=3.0,
            selected_features=feats,
            out_name=f"data_{name}.h5",
        )
        print(f"Wrote new H5: {out_dir / f'data_{name}.h5'}")

All H5 files already exist. Skipping feature selection and H5 generation.


In [ ]:
# Config
FEATURESETS = [
    ("HSIC", h5_path_HSIC),
    ("PCC",  h5_path_PCC),
    ("MI",   h5_path_MI),
]
PATCH_SIZE   = 128
STRIDE       = 64
MIN_VALID_FR = 0.80
VAL_FRACTION = 0.10
LR           = 1e-3
EPOCHS       = 50
BATCH_SIZE   = 4
EPS          = 1e-6

# Config for UNet artifacts
ARTIFACTS_DIR_UNET = "UNet_weights_and_norm"
os.makedirs(ARTIFACTS_DIR_UNET, exist_ok=True)

FINAL_NORM_UNET    = os.path.join(ARTIFACTS_DIR_UNET, "best_norm_stats.npz")
FINAL_WEIGHTS_UNET = os.path.join(ARTIFACTS_DIR_UNET, "unet_best.weights.h5")
FINAL_TAG_UNET     = os.path.join(ARTIFACTS_DIR_UNET, "unet_best.tag.txt")

results_unet = []  # (name, best_val_loss, train_time_min, tmp_norm_path, tmp_weights_path)

for name, h5_path in FEATURESETS:
    # Split data (per-feature set)
    (X_train, y_train, m_train), (X_val, y_val, m_val), _ = DS.DL_split(
        h5_path=str(h5_path),
        holdout_aoi=holdout_aoi,
        val_fraction=VAL_FRACTION,
        patch_size=PATCH_SIZE,
        stride=STRIDE,
        min_valid_frac=MIN_VALID_FR,
    )
    print(f"{name} Shapes: X_train {X_train.shape}, y_train {y_train.shape}, X_val {X_val.shape}, y_val {y_val.shape}")

    # Train-only stats (NaN-safe) + normalize
    tmp_norm_path = os.path.join(ARTIFACTS_DIR_UNET, f"{name}_norm_stats.tmp.npz")
    mean = np.nanmean(X_train, axis=(0, 1, 2), keepdims=True).astype("float32")
    std  = (np.nanstd (X_train, axis=(0, 1, 2), keepdims=True) + EPS).astype("float32")
    std  = np.where(std < EPS, 1.0, std).astype("float32")
    np.savez(tmp_norm_path, mean=mean, std=std)

    with np.load(tmp_norm_path) as f:
        mu, sigma = f["mean"], f["std"]
    X_train = np.nan_to_num(X_train, nan=mu, posinf=mu, neginf=mu).astype("float32")
    X_val   = np.nan_to_num(X_val,   nan=mu, posinf=mu, neginf=mu).astype("float32")
    X_train_n = ((X_train - mu) / sigma).astype("float32")
    X_val_n   = ((X_val   - mu) / sigma).astype("float32")

    # Labels + weights
    y_train_f, w_train = ARCH.fill_nan_and_mask(y_train)
    y_val_f,   w_val   = ARCH.fill_nan_and_mask(y_val)
    w_train_4d = w_train[..., None].astype("float32")
    w_val_4d   = w_val[..., None].astype("float32")

    # Build & compile U-Net
    model = ARCH.unet(input_shape=X_train_n.shape[1:], base_filters=32)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
        loss=tf.keras.losses.Huber(delta=1.0),
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(name="MAE"),
            tf.keras.metrics.RootMeanSquaredError(name="RMSE"),
        ],
    )

    # Callbacks monitoring val_loss (Huber)
    tmp_weights = os.path.join(ARTIFACTS_DIR_UNET, f"unet_{name}.tmp.weights.h5")
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            tmp_weights, monitor="val_loss", mode="min",
            save_best_only=True, save_weights_only=True, verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=10, restore_best_weights=True, verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", mode="min",
            factor=0.5, patience=5, min_lr=1e-5, verbose=1
        ),
    ]

    # Train
    start_time = time.time()
    hist = model.fit(
        X_train_n, y_train_f,
        sample_weight=w_train_4d,
        validation_data=(X_val_n, y_val_f, w_val_4d),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1,
    )
    train_time = (time.time() - start_time) / 60.0

    # Best epoch by val_loss, also report MAE/RMSE there
    best_idx       = int(np.argmin(hist.history["val_loss"]))
    best_val_loss  = float(hist.history["val_loss"][best_idx])
    best_val_mae   = float(hist.history["val_MAE"][best_idx])
    best_val_rmse  = float(hist.history["val_RMSE"][best_idx])

    print(f"{name}: best val_loss={best_val_loss:.6f} @ epoch {best_idx+1} "
          f"(val_MAE={best_val_mae:.4f}, val_RMSE={best_val_rmse:.4f}); time={train_time:.2f} min")

    results_unet.append((name, best_val_loss, train_time, tmp_norm_path, tmp_weights))



HSIC Shapes: X_train (576, 128, 128, 10), y_train (576, 128, 128, 1), X_val (64, 128, 128, 10), y_val (64, 128, 128, 1)
Epoch 1/50
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - MAE: 0.5970 - loss: 0.2743
Epoch 1: val_loss improved from inf to 0.87574, saving model to UNet_weights_and_norm\unet_HSIC.tmp.weights.h5
144/144 ━━━━━━━━━━━━━━━━━━━━ 58s 354ms/step - MAE: 0.5959 - loss: 0.2735 - val_MAE: 1.2728 - val_loss: 0.8757 - learning_rate: 0.0010
Epoch 2/50
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - MAE: 0.3553 - loss: 0.0928
Epoch 2: val_loss did not improve from 0.87574
144/144 ━━━━━━━━━━━━━━━━━━━━ 48s 336ms/step - MAE: 0.3552 - loss: 0.0928 - val_MAE: 2.0201 - val_loss: 1.6888 - learning_rate: 0.0010
Epoch 3/50
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - MAE: 0.3271 - loss: 0.0821
Epoch 3: val_loss improved from 0.87574 to 0.09055, saving model to UNet_weights_and_norm\unet_HSIC.tmp.weights.h5
144/144 ━━━━━━━━━━━━━━━━━━━━ 48s 334ms/step - MAE: 0.3271 - loss: 0.0822 - val_MAE: 0.3253

In [ ]:
# Pick the global U-Net winner by validation loss (Huber)
results_sorted = sorted(results_unet, key=lambda t: t[1])  # lower is better
winner_name, winner_loss, winner_time, winner_norm, winner_weights = results_sorted[0]

print("\n[U-Net] Validation leaderboard (lower is better)")
for rank, (n, v_loss, tmin, _, _) in enumerate(results_sorted, 1):
    print(f"{rank}. {n:>8s}  val_loss={v_loss:.6f}  time={tmin:.2f}m")
print(f"\n[U-Net] WINNER: {winner_name} (val_loss={winner_loss:.6f})")

# Clean previous finals if exist
for p in [FINAL_NORM_UNET, FINAL_WEIGHTS_UNET, FINAL_TAG_UNET]:
    if os.path.exists(p):
        os.remove(p)

# Move winning artifacts to final names
shutil.move(winner_norm,   FINAL_NORM_UNET)
shutil.move(winner_weights, FINAL_WEIGHTS_UNET)
with open(FINAL_TAG_UNET, "w") as f:
    f.write(f"winner={winner_name}\nval_loss={winner_loss:.6f}\n")

print(f"\nSaved winner norm stats -> {FINAL_NORM_UNET}")
print(f"Saved winner weights   -> {FINAL_WEIGHTS_UNET}")
print(f"Wrote tag file         -> {FINAL_TAG_UNET}")



Validation leaderboard (lower is better)
1. HSIC  val_loss=0.039178  time=32.82s
2.   MI  val_loss=0.043043  time=16.62s
3.  PCC  val_loss=0.063271  time=10.61s

WINNER: HSIC (val_loss=0.039178)

Saved winner norm stats -> UNet_weights_and_norm\best_norm_stats.npz
Saved winner weights -> UNet_weights_and_norm\unet_best.weights.h5
Wrote tag file -> UNet_weights_and_norm\unet_best.tag.txt


In [ ]:
# U-Net with ALL features (Ablation study)
name = "ALL_UNET_REPORT"

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Ensure ALL-features H5 exists
h5_path_ALL = ROOT / "data" / "h5_dir" / "ALL" / "data_ALL.h5"
if not h5_path_ALL.exists():
    (h5_dir / "ALL").mkdir(parents=True, exist_ok=True)
    DL.build_h5(
        data_dir=str(ROOT / "data" / "tif_files"),
        out_dir=str(h5_dir / "ALL"),
        write_mask=True,
        selected_features=FEATURE_NAMES,  # use your full list; swap to FEATURE_SET if that's your alias
        out_name="data_ALL.h5",
    )
    print(f"Wrote ALL-features H5: {h5_path_ALL}")
else:
    print(f"ALL-features H5 already exists: {h5_path_ALL}")

# Split (dev train/val + holdout AOI)
(X_train, y_train, m_train), (X_val, y_val, m_val), _ = DS.DL_split(
    h5_path=str(h5_path_ALL),
    holdout_aoi=HOLDOUT_AOI,
    val_fraction=VAL_FRACTION,
    patch_size=PATCH_SIZE,
    stride=STRIDE,
    min_valid_frac=MIN_VALID_FR,
)
print(f"{name} Shapes: X_train {X_train.shape}, y_train {y_train.shape}, X_val {X_val.shape}, y_val {y_val.shape}")

# NaN-safe normalization from train only
mu  = np.nanmean(X_train, axis=(0, 1, 2), keepdims=True).astype("float32")
std = (np.nanstd (X_train, axis=(0, 1, 2), keepdims=True) + EPS).astype("float32")
std = np.where(std < 1e-6, 1.0, std).astype("float32")

X_train = np.nan_to_num(X_train, nan=mu, posinf=mu, neginf=mu).astype("float32")
X_val   = np.nan_to_num(X_val,   nan=mu, posinf=mu, neginf=mu).astype("float32")
X_train_n = ((X_train - mu) / std).astype("float32")
X_val_n   = ((X_val   - mu) / std).astype("float32")

# Labels + masks
y_train_f, w_train = ARCH.fill_nan_and_mask(y_train)
y_val_f,   w_val   = ARCH.fill_nan_and_mask(y_val)
w_train_4d = w_train[..., None].astype("float32")
w_val_4d   = w_val[..., None].astype("float32")

# Build & compile U-Net
model = ARCH.unet(input_shape=X_train_n.shape[1:], base_filters=32)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[
        tf.keras.metrics.MeanAbsoluteError(name="MAE"),
        tf.keras.metrics.RootMeanSquaredError(name="RMSE"),
    ],
)

# Train with val_loss monitoring; no checkpoints saved
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", mode="min", patience=10, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min", factor=0.5, patience=5, min_lr=1e-5, verbose=1
    ),
]

start_time = time.time()
hist = model.fit(
    X_train_n, y_train_f,
    sample_weight=w_train_4d,
    validation_data=(X_val_n, y_val_f, w_val_4d),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1,
)
train_time = (time.time() - start_time) / 60.0

# Report best-loss epoch with MAE/RMSE
best_idx      = int(np.argmin(hist.history["val_loss"]))
best_val_loss = float(hist.history["val_loss"][best_idx])
best_val_mae  = float(hist.history["val_MAE"][best_idx])
best_val_rmse = float(hist.history["val_RMSE"][best_idx])

print(f"\n[{name}] best val_loss={best_val_loss:.6f} @ epoch {best_idx+1} "
      f"(val_MAE={best_val_mae:.4f}, val_RMSE={best_val_rmse:.4f}); "
      f"train_time={train_time:.2f} min")


ALL-features H5 already exists: c:\Users\mathi\Documents\Paper2\ML_SD\data\h5_dir\ALL\data_ALL.h5
ALL_UNET_REPORT Shapes: X_train (580, 128, 128, 27), y_train (580, 128, 128, 1), X_val (64, 128, 128, 27), y_val (64, 128, 128, 1)
Epoch 1/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 64s 371ms/step - MAE: 0.4828 - RMSE: 0.6702 - loss: 0.1848 - val_MAE: 0.6179 - val_RMSE: 0.7559 - val_loss: 0.2463 - learning_rate: 0.0010
Epoch 2/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 53s 364ms/step - MAE: 0.3444 - RMSE: 0.4750 - loss: 0.0888 - val_MAE: 0.3256 - val_RMSE: 0.4996 - val_loss: 0.0903 - learning_rate: 0.0010
Epoch 3/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 52s 361ms/step - MAE: 0.3384 - RMSE: 0.4579 - loss: 0.0858 - val_MAE: 0.3312 - val_RMSE: 0.4736 - val_loss: 0.0882 - learning_rate: 0.0010
Epoch 4/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 52s 360ms/step - MAE: 0.3255 - RMSE: 0.4485 - loss: 0.0775 - val_MAE: 0.2907 - val_RMSE: 0.4346 - val_loss: 0.0726 - learning_rate: 0.0010
Epoch 5/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 53s 364ms/step 